In [1]:
import pandas as pd
from pathlib import Path
import ast
from ast import literal_eval
#from updates_join_annotations import open_df_from_tsv
#import updates_get_average_scores_per_label as scores


In [20]:
def clean_data(filepath):
    # Read in file
    df = pd.read_excel(filepath, dtype={'tokens': list, 'annotators':set}, index_col=0)
    # Sort values and reset index to allow looping later
    df.sort_values(by=['file_id', 'start_index', 'end_index', 'Label'], inplace=True)
    df.reset_index(inplace=True, drop=True)
    # Drop repeated columns
    df.drop(['Unnamed: 0.1', 'Unnamed: 1', 'Unnamed: 2'], axis=1, inplace=True)
    
    # Read in file and set data type for two columns
    df['tokens'] = df['tokens'].apply(literal_eval)
    df['annotators'] = df['annotators'].apply(literal_eval)
    
    return df

In [21]:
# def main():
# filepath = '../sample_data/label_level_df.xlsx'
filepath = '../../Data/label_level_df_sorted_cleaned.xlsx'

df = clean_data(filepath)

# To do: Something strange happens in row 1289. Token length = 3, start token = 7, end token = 83, len annotator set = 4, annotator count = 1.
# Also row 1375

In [22]:
df.head()

,Label1,Label2,Label,file_id,start_index,end_index,tokens,annotators,annotator_count
0,view\,Patient,view\Patient,2503,48,54,"[Verteld, zelf, dat, hij, erg, productief, is]",{katsburg},1
1,.B152: Stemming,NaN,.B152: Stemming,2503,96,98,"[Mw, was, emotioneel]",{vanderpas},1
2,STM 1,NaN,STM 1,2503,96,98,"[Mw, was, emotioneel]",{vanderpas},1
3,.B152: Stemming,NaN,.B152: Stemming,2503,97,98,"[was, emotioneel]",{opsomer},1
4,.B152: Stemming,NaN,.B152: Stemming,2503,98,98,[emotioneel],"{avelli, vervaart, meskers, katsburg, bos, swa...",6


In [23]:


#file_id = 2345
#label = 'STM0'
label_dict = dict()

# Loop through file_ids (change later)
for file_id in set(df['file_id']):
#for file_id in [2503]:
    # Create filtered df
    df_filtered = df[df['file_id'] == file_id]
    # Loop through labels and create further filtered df
    for label in set(df_filtered['Label']):
                
        #print(file_id, label)
        # df_label has the selection of the rows with the same label
        df_label = df_filtered[df_filtered['Label'] == label]
        df_label.reset_index(inplace=True, drop=True)
        
        try:
            # Set start and end index values
            start_index = df_label['start_index'][0]
            end_index = df_label['end_index'][0]
        except:
            continue
        
        #print('first annotation:', start_index, end_index)
        firstannotation = True
        counter = len(label_dict)+1
        try:
            annos = df_label['annotators'].iloc[1]
        except:
            continue
        
        label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': df_label.head(1)['tokens'],
                                           'annotators': annos}
        current_range = range(start_index, end_index+1)

        if df_label.shape[0] == 1:
            #print('There is only one annotation for this label so we are done.')
            # If only one tag of label in this file, we are done
            continue
 

        # There are multiple annotations so we need to check their spans and merge
        # label through rest of df
        for index, row in df_label[1:].iterrows():
            # Note: df which it is looping over contains only data from one file from one label

            start_temp = row['start_index']
            end_temp = row['end_index']
            #print('temp', start_temp, end_temp)

            # Since the df_label is sorted on start & end index, only a few cases need to be checked
            # If the temporary start vallue is in the range of the saved start value
            if start_temp in current_range:
                #print('included')
                #print('counter', counter)
                if end_temp not in current_range: # And the temp end value is not, then update the end value
                    #print('updated')
                    end_index = end_temp
                    ### we need to adapt the range
                    current_range = range(start_index, end_index+1)
                # If the temp end value is also in the range then do nothing and move on.
                ### we update the info for this annotation
                if counter in label_dict:
                    #print('counter in label_dict and adding annotators', start_index, end_index)
                    label_dict[counter]['end_index'] = end_index
                    currentannos = label_dict[counter]['annotators']
                    currentannos.update(row['annotators'])
                    label_dict[counter]['annotators'] = currentannos
                    #print('Extended annotators:',label_dict[counter]['annotators'])
                else:
                    #print('counter not in label_dict', start_index, end_index)
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
                
            else:
                
                ### We have a new first annotation with start and range
                start_index = start_temp
                end_index = end_temp
                current_range = range(start_index, end_index+1)
                counter = len(label_dict)+1
                label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
                
            # For last entry
            if index == len(df_label)-1:
                #print('last, stored')
                if counter in label_dict:
                    label_dict[counter]['end_index'] = end_index
                    currentannos = label_dict[counter]['annotators']
                    currentannos.update(row['annotators'])
                    label_dict[counter]['annotators'] = currentannos

                    #print('Extended annotators:',label_dict[counter]['annotators'])
                   
                else: 
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
    


In [24]:
label_dict

{1: {'file_id': 4100,
  'start_index': 517,
  'end_index': 520,
  'label': 'FAC 2',
  'tokens': 0    [niet, zonder, steun]
  Name: tokens, dtype: object,
  'annotators': {'bos'}},
 2: {'file_id': 4100,
  'start_index': 507,
  'end_index': 507,
  'label': 'type\\Implicit',
  'tokens': 0    [mobiliseren]
  Name: tokens, dtype: object,
  'annotators': {'swartjes'}},
 3: {'file_id': 4100,
  'start_index': 556,
  'end_index': 558,
  'label': 'type\\Implicit',
  'tokens': ['onvermogen', 'te', 'mobiliseren'],
  'annotators': {'swartjes'}},
 4: {'file_id': 4100,
  'start_index': 263,
  'end_index': 282,
  'label': 'view\\Patient',
  'tokens': 0    [Patient, geeft, aan, op, linkerzij, gevallen,...
  Name: tokens, dtype: object,
  'annotators': {'katsburg'}},
 5: {'file_id': 4100,
  'start_index': 510,
  'end_index': 513,
  'label': 'view\\Patient',
  'tokens': ['pijn', 'aan', 'in', 'linkerheup'],
  'annotators': {'katsburg'}},
 6: {'file_id': 4100,
  'start_index': 515,
  'end_index': 524,
  'l

In [6]:
#### The next function could be differentiated per label
#- gemiddelde aantal gedeelde annotaties per label
#- hoeveel annotaties per student
#- hoe vaak is student outlier
#- confusion matrix
from collections import Counter

label_agreement = {}
labels = []
annotators = []
annotation_total = 0
for key, value in label_dict.items():
    label = label_dict[key]['label']
    labels.append(label)
    key_annotators= label_dict[key]['annotators']
    for anno in key_annotators:
        annotators.append(anno)
    if label in label_agreement:
        label_agreement[label]+=len(key_annotators)
    else:
        label_agreement[label]=len(key_annotators)
    annotation_total += len(key_annotators)


print('Annotations per label', Counter(labels))
print('Annotations per annotator',Counter(annotators))
print('Total annotations:', len(label_dict.items()))
print('Total student annotations:', annotation_total)
print('Average agreement of annotations', annotation_total/(len(label_dict.items())))

# for key, value in label_agreement.items():
#     print(key, value)

Annotations per label Counter({'.B152: Stemming': 119, 'view\\Patient': 100, 'STM 1': 95, 'type\\Implicit': 91, '.D450: Lopen en zich verplaatsen': 62, 'stm\\reaction': 61, 'FAC 4': 53, '.B455: Inspanningstolerantie': 41, 'view\\Third party': 40, 'STM 3': 30, 'STM 0': 27, 'STM 2': 22, 'INS 2': 17, 'FAC 2': 16, 'FAC 1': 15, 'INS 1': 15, '.D840-859: Beroep en werk': 14, 'FAC 3': 14, 'FAC 0': 11, 'FAC 5': 10, 'INS 3': 5, 'INS 5': 3, 'BER 0': 3, 'INS 4': 2, 'STM 4': 2, 'BER 4': 1, 'BER 1': 1})
Annotations per annotator Counter({'katsburg': 420, 'swartjes': 374, 'meskers': 369, 'avelli': 281, 'vervaart': 252, 'bos': 247, 'vanderpas': 162, 'opsomer': 153})
Total annotations: 870
Total student annotations: 2258
Average agreement of annotations 2.5954022988505745


In [8]:
# Function differentiated per label

labels = []
annotators = []
annotation_total = 0
for key, value in label_dict.items():
    #label naar keus invullen
    if label_dict[key]['label'] == 'STM 4':
        label = label_dict[key]['label']
        labels.append(label)
        key_annotators=label_dict[key]['annotators']
        for anno in key_annotators:
            annotators.append(anno)
        annotation_total += len(key_annotators)
        
        
print('Annotations per label', Counter(labels))
print()
print('Annotations per annotator',Counter(annotators))
print()
print('Total annotations:', len(labels))
print()
print('Total student annotations:', annotation_total)
print()
print('Average agreement of annotations', annotation_total/len(labels))

Annotations per label Counter({'STM 4': 3})

Annotations per annotator Counter({'meskers': 2, 'bos': 1, 'vervaart': 1, 'katsburg': 1, 'opsomer': 1})

Total annotations: 3

Total student annotations: 6

Average agreement of annotations 2.0


In [9]:
df_soft = pd.DataFrame.from_dict(label_dict, orient='index')

In [10]:
df_soft.head()

,file_id,start_index,end_index,label,tokens,annotators
1,4100,517,520,FAC 2,"0 [niet, zonder, steun] Name: tokens, dtype...",{bos}
2,4100,157,158,.B455: Inspanningstolerantie,"0 [Patiente, fietste] Name: tokens, dtype: ...",{swartjes}
3,4100,507,507,.D450: Lopen en zich verplaatsen,"0 [mobiliseren] Name: tokens, dtype: object","{bos, avelli, katsburg, swartjes, meskers, van..."
4,4100,515,520,.D450: Lopen en zich verplaatsen,"[Kan, hierdoor, niet, zonder, steun, lopen]","{bos, avelli, katsburg, swartjes, meskers, van..."
5,4100,557,558,.D450: Lopen en zich verplaatsen,"[te, mobiliseren]","{katsburg, swartjes}"


In [11]:
# add row to df with a number for the amount of annotators per annotation

list_lens = []
for index, item in enumerate(df_soft['annotators']):
    if type(item) == str:
        item2 = item.split(' ')
        list_lens.append(len(item2))
    else: 
        list_lens.append(len(item))
        
#print(list_lens)    
    
df_soft['number_of_annotators'] = list_lens
    

In [12]:
df_soft.head()

,file_id,start_index,end_index,label,tokens,annotators,number_of_annotators
1,4100,517,520,FAC 2,"0 [niet, zonder, steun] Name: tokens, dtype...",{bos},1
2,4100,157,158,.B455: Inspanningstolerantie,"0 [Patiente, fietste] Name: tokens, dtype: ...",{swartjes},1
3,4100,507,507,.D450: Lopen en zich verplaatsen,"0 [mobiliseren] Name: tokens, dtype: object","{bos, avelli, katsburg, swartjes, meskers, van...",6
4,4100,515,520,.D450: Lopen en zich verplaatsen,"[Kan, hierdoor, niet, zonder, steun, lopen]","{bos, avelli, katsburg, swartjes, meskers, van...",6
5,4100,557,558,.D450: Lopen en zich verplaatsen,"[te, mobiliseren]","{katsburg, swartjes}",2


In [13]:
df_soft.to_excel('softboundary_output_file_newest3.xlsx')

In [14]:
df_soft

,file_id,start_index,end_index,label,tokens,annotators,number_of_annotators
1,4100,517,520,FAC 2,"0 [niet, zonder, steun] Name: tokens, dtype...",{bos},1
2,4100,157,158,.B455: Inspanningstolerantie,"0 [Patiente, fietste] Name: tokens, dtype: ...",{swartjes},1
3,4100,507,507,.D450: Lopen en zich verplaatsen,"0 [mobiliseren] Name: tokens, dtype: object","{bos, avelli, katsburg, swartjes, meskers, van...",6
4,4100,515,520,.D450: Lopen en zich verplaatsen,"[Kan, hierdoor, niet, zonder, steun, lopen]","{bos, avelli, katsburg, swartjes, meskers, van...",6
5,4100,557,558,.D450: Lopen en zich verplaatsen,"[te, mobiliseren]","{katsburg, swartjes}",2
...,...,...,...,...,...,...,...
872,3581,432,435,STM 1,"[Ziet, er, tegen, op]",{avelli},1
873,3581,170,172,STM 2,"0 [Niet, veel] Name: tokens, dtype: object","{meskers, avelli, vervaart, katsburg}",4
874,3581,16,23,view\Patient,"0 [Pt, weet, niet, meer, wat, hij, leuk, vi...",{meskers},1
875,3581,161,166,view\Patient,"[Voelt, zich, over, het, algemeen, gemakkelijk]",{meskers},1


In [15]:
# extract information about the annotations made by only one annotator
# label count

list_labels = []

for index, item in enumerate(df_soft['number_of_annotators']):
    if item == 1:
        list_labels.append(df_soft.iloc[index]['label'])
        
Counter(list_labels)

list_annotators = []

    

In [16]:
# extract information about the annotations made by only one annotator
# annotator count

list_annotators = []

for index, item in enumerate(df_soft['number_of_annotators']):
    if item == 1:
        list_annotators.append(str(df_soft.iloc[index]['annotators']))
        
Counter(list_annotators)

# print()
# print(list_annotators)



Counter({"{'bos'}": 33,
         "{'swartjes'}": 76,
         "{'katsburg'}": 142,
         "{'meskers'}": 82,
         "{'avelli'}": 44,
         "{'vervaart'}": 20,
         "{'vanderpas'}": 19,
         "{'opsomer'}": 14})

In [17]:
# extract labels and spans of annotations made by only one annotator for a certain annotator

list_tuples = []
for index, item in enumerate(df_soft['number_of_annotators']):
    # fille in name of annotator at end of the next line
    if item == 1 and df_soft.iloc[index]['annotators'] == {'swartjes'}:
        list_tuples.append((df_soft.iloc[index]['label'], df_soft.iloc[index]['tokens']))

In [18]:
print(len(list_tuples))

for item in list_tuples:
    print(item)

76
('.B455: Inspanningstolerantie', 0    [Patiente, fietste]
Name: tokens, dtype: object)
('type\\Implicit', 0    [mobiliseren]
Name: tokens, dtype: object)
('type\\Implicit', ['onvermogen', 'te', 'mobiliseren'])
('view\\Third party', 0    [Zijn, vrouw, is, kwaad, op, hem, dat, ze, nie...
Name: tokens, dtype: object)
('view\\Third party', ['Hij', 'is', 'volledig', 'overbelast', 'als', 'mantelzorger', 'en', 'heeft', 'weinig', 'ziekte', 'inzicht'])
('view\\Third party', ['Hij', 'slaapt', 'al', 'jaren', 'heel', 'slecht', ',', 'hierdoor', 'is', 'hij', 'overdag', 'vermoeid'])
('view\\Third party', ['Hij', 'is', 'bang', 'dat', 'hij', 'niet', 'meer', 'mag', 'autorijden'])
('.B455: Inspanningstolerantie', ['aan', 'en', 'uitkleden'])
('view\\Patient', ['Zijn', 'vrouw', 'is', 'kwaad', 'op', 'hem', 'dat', 'ze', 'niet', 'mag', 'autorijden'])
('view\\Patient', ['Belt', 'dagelijks', ',', 'zij', 'wordt', 'dan', 'boos', 'en', 'eist', 'de', 'autosleutels'])
('STM 1', ['vreest'])
('STM 1', 0    [onrust/

In [24]:
# extract vocabulary per label
token_dict = {}
for index, item in enumerate(df_soft['label']):
    # fille in name of annotator at end of the next line
    for tokens in df_soft.iloc[index]['tokens']:
        if type(tokens)==str:
            tokens = [tokens]
        for token in tokens:
            if token:
                if token in token_dict:
                    token_dict[token].append(item)
                else:
                    token_dict[token] = [item]

for key, value in token_dict.items():
    print(key, Counter(value))


niet Counter({'view\\Patient': 38, 'view\\Third party': 15, 'STM 1': 8, 'type\\Implicit': 8, '.D450: Lopen en zich verplaatsen': 6, 'STM 2': 6, 'FAC 1': 5, 'stm\\reaction': 5, '.B152: Stemming': 5, 'FAC 2': 3, 'INS 2': 3, 'FAC 0': 3, 'STM 0': 2, 'INS 1': 2, 'BER 0': 2, '.B455: Inspanningstolerantie': 2, '.D840-859: Beroep en werk': 1, 'FAC 3': 1, 'FAC 4': 1})
zonder Counter({'FAC 4': 3, 'type\\Implicit': 2, 'FAC 2': 1, '.D450: Lopen en zich verplaatsen': 1, 'FAC 1': 1, '.B152: Stemming': 1, 'view\\Patient': 1, 'stm\\reaction': 1})
steun Counter({'FAC 2': 1, '.D450: Lopen en zich verplaatsen': 1, 'FAC 1': 1, 'FAC 0': 1, 'type\\Implicit': 1})
Patiente Counter({'view\\Third party': 2, '.B455: Inspanningstolerantie': 1, 'type\\Implicit': 1, 'STM 1': 1, 'STM 2': 1, 'view\\Patient': 1})
fietste Counter({'.B455: Inspanningstolerantie': 1})
mobiliseren Counter({'type\\Implicit': 4, '.D450: Lopen en zich verplaatsen': 3})
Kan Counter({'.D450: Lopen en zich verplaatsen': 3, 'type\\Implicit': 2, 